01. Initializations

In [11]:
import pandas as pd
import sys
import json
import os
import datetime


root_folder = os.path.join("/", "RanD", "CREMEv2_Result", "20230310", "logs_working")
scenario = ["mirai", "disk_wipe", "ransomware", "resource_hijacking", "end_point_dos"]
scenario_folder = ["01_mirai", "02_disk_wipe", "03_ransomware", "04_resource_hijacking", "05_end_point_dos"]
syslog_folder = "syslog"
label_folder = "label_syslog"
filename = "dataset_generation.log"
label = {'mirai': [1, 2, 3, 5, 9, 11, 12, 13],
         'disk_wipe': [1, 2, 4, 6, 8, 14],
         'ransomware': [1, 2, 4, 6, 7, 12, 15],
         'resource_hijacking': [1, 2, 4, 6, 8, 12, 16],
         'end_point_dos': [1, 2, 4, 8, 10, 12, 17]
         }

ips = {"controller": ["192.168.56.111"],
       "data-logger": ["192.168.56.121"],
       "attacker": ["192.168.56.131"],
       "non-vulnerable": ["192.168.56.141", "192.168.56.142"],
       "vulnerable": ["192.168.56.151"],
       "malicious": ["192.168.56.161"],
       "benign": ["192.168.56.171"],
       "target": ["192.168.56.181"]
       }

2. Repairing the label

In [6]:
df = pd.read_csv(os.path.join(root_folder, label_folder, "original_label_syslog.csv"))
del_list = ['Tactic', 'Technique', 'SubTechnique']
result_abs_path = os.path.join(root_folder, label_folder, "modified_label")
# print(ips["benign"][0])
df.Label[df.HostName=="192.168.56.171"] = 0
df.Label_lifecycle[df.HostName=="192.168.56.171"] = 0
df['HostName'] = df['HostName'].replace('192.168.56.171', 'benign-server')
# print(df)
df = df.drop(columns=del_list)

if not os.path.exists(result_abs_path):
    os.makedirs(result_abs_path)
df.to_csv(os.path.join(result_abs_path, "original_relabel_syslog.csv"), index=False)

/tmp/ipykernel_303462/2935807126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Label[df.HostName=="192.168.56.171"] = 0
/tmp/ipykernel_303462/2935807126.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Label_lifecycle[df.HostName=="192.168.56.171"] = 0


In [14]:
col_list = ['HostName', 'EventTemplate', 'Timestamp', 'Label', 'Label_lifecycle']
one_hot_col_list = ['EventTemplate']

df = pd.read_csv(os.path.join(root_folder,  label_folder, "modified_label", "original_relabel_syslog.csv"), usecols=col_list)
labels = df['Label'].unique()

df = pd.get_dummies(df, columns=one_hot_col_list)

hostnames = df['HostName'].unique()

df_count_vector = pd.DataFrame()

df_machines = []
unique_timestamp_machines = []
for hostname in hostnames:
    df_machines.append(df[df['HostName'] == hostname])
    
for df_machine in df_machines:
    unique_timestamps = list(set(df_machine['Timestamp']))
    unique_timestamp_machines.append(unique_timestamps)

del_col_list = ['HostName', 'Timestamp']
for i in range(len(df_machines)):
    df_machine = df_machines[i]
    tmp_unique_timestamps = unique_timestamp_machines[i]
    for tmp_timestamp in tmp_unique_timestamps:
        tmp_df = df_machine[df_machine['Timestamp'] == tmp_timestamp]
        tmp_df = tmp_df.drop(columns=del_col_list)
        classes_num = {}
        for j in labels:
            class_num = len(tmp_df[tmp_df['Label'] == j])
            if j != 0 and class_num > 0:
                classes_num[j] = class_num
            sum_one_hot = tmp_df.sum()
            
        max_num = 0
        max_label = 0
        for key in classes_num:
            if classes_num[key] > max_num:
                max_num = classes_num[key]
                max_label = key
                
        sum_one_hot = pd.DataFrame(sum_one_hot).transpose()
        sum_one_hot['Label'] = max_label
        sum_one_hot['Timestamp'] = tmp_timestamp
        df_count_vector = pd.concat([df_count_vector, sum_one_hot], ignore_index=True)


if not os.path.exists(result_abs_path):
    os.makedirs(result_abs_path)
df.to_csv(os.path.join(result_abs_path, "relabel_syslog.csv"), index=False)

In [15]:
df = pd.read_csv(os.path.join(root_folder,  label_folder, "modified_label", "relabel_syslog.csv"))
df.head()

,HostName,Timestamp,Label,Label_lifecycle,"EventTemplate_""CLUR / HTTP/1.1"" 501 496 ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""","EventTemplate_""FHWL / HTTP/1.1"" 501 496 ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""","EventTemplate_""GET /chat/read_log.php HTTP/1.1"" 404 <*> ""about:blank"" ""Node.js (linux; U; rv:v4.9.1) AppleWebKit/537.36 (KHTML, like Gecko)""","EventTemplate_""GET <*> <*> 200 <*> ""-"" <*>","EventTemplate_""GET <*> HTTP/1.1"" <*> <*> ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""","EventTemplate_""KDUN / HTTP/1.1"" 501 496 ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""",...,EventTemplate_pam_unix(samba:session): session closed for user nobody,EventTemplate_pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=<*> user=root,EventTemplate_pam_unix(sshd:session): session closed for user root,EventTemplate_pam_unix(sshd:session): session opened for user root by (uid=0),EventTemplate_statistics: max cache size <*> at Mar 6 <*>,EventTemplate_statistics: max connection <*> <*> for <*> at Mar 6 <*>,EventTemplate_warning: TLS library problem: <*> <*> <*>,EventTemplate_warning: TLS library problem: error:14094410:SSL routines:SSL3_READ_BYTES:sslv3 alert handshake failure:s3_pkt.c:1278:SSL alert number 40:,EventTemplate_warning: non-SMTP command from unknown[<*>]: <*>,EventTemplate_warning: unknown[<*>]: SASL NTLM authentication failed: Invalid authentication mechanism
0,target-server,1.678093e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,target-server,1.678093e+09,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,target-server,1.678093e+09,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,target-server,1.678093e+09,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,target-server,1.678093e+09,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
